In [1]:
from lightautoml.dataset.seq_np_pd_dataset import SeqNumpyPandasDataset
from lightautoml.reader.seq import IDSInd, TopInd
from lightautoml.reader.base import DictToPandasSeqReader
from lightautoml.tasks import Task
import pandas as pd
import numpy as np
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM

from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBSeqSimpleFeatures, LGBMultiSeqSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline

import numpy as np

In [2]:
import pandas as pd
import numpy as np

DATA_FOLDER = '../../data/small_new/relational_data'

fulfilment_center_info = pd.read_csv(f'{DATA_FOLDER}/fulfilment_center_info.csv')
meal_info = pd.read_csv(f'{DATA_FOLDER}/meal_info.csv')
train = pd.read_csv(f'{DATA_FOLDER}/train.csv')

val = 0.05


In [3]:
seq_params = {
              'fulfilment_center_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'center_id', 'to_id': 'center_id'},
                          },
              'meal_info':{'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'meal_id', 'to_id': 'meal_id'},
                          },
              }
seq_data = {'fulfilment_center_info': fulfilment_center_info[['center_id',
                                                              'city_code',
                                                              'region_code',
                                                              'op_area']],
       'meal_info': meal_info              
       }

X_train = {'plain':train.sample(frac=0.05).reset_index(drop=True) , 
           'seq': seq_data
          }

task = Task('reg', metric='mae')

roles={'target': 'num_orders'}




In [4]:
task = Task('reg', metric='mae')
roles={'target': 'num_orders'}
reader = DictToPandasSeqReader(task=task, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

In [5]:
%%time

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)


[19:10:43] Feats was rejected during automatic roles guess: []
[19:10:43] Layer 1 train process start. Time left 9999999995.94 secs
[19:10:43] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[19:10:43] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[19:10:43] Training until validation scores don't improve for 100 rounds
[19:10:44] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[19:10:44] Training until validation scores don't improve for 100 rounds
[19:10:45] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[19:10:45] Training until validation scores don't improve for 100 rounds
[19:10:46] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[19:10:46] Training until validation scores don't improve for 100 rounds
[19:10:47] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[19:10:47] Training until validation scores don't improve for 100 rounds
[19:10:48] Fitting Lvl_0_Pipe_0_Mod_0_Lig

In [6]:
from lightautoml.dataset.gpu.gpu_dataset import SeqCudfDataset
from lightautoml.dataset.gpu.gpu_dataset import SeqDaskCudfDataset
from lightautoml.reader.gpu.seq_gpu import IDSIndGPU, TopIndGPU
from lightautoml.reader.gpu.seq_reader_gpu import DictToCudfSeqReader
from lightautoml.reader.gpu.seq_reader_gpu import DictToDaskCudfSeqReader

from lightautoml.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml.ml_algo.gpu.boost_cb_gpu import BoostCBGPU

from lightautoml.pipelines.features.gpu.lgb_pipeline_gpu import LGBMultiSeqSimpleFeaturesGPU

import cudf
import dask_cudf
import cupy as cp

In [7]:
task = Task('reg', metric='mae', device='gpu')
reader = DictToCudfSeqReader(task=task, seq_params=seq_params, n_jobs=1)

feats = LGBMultiSeqSimpleFeaturesGPU()
model = BoostCBGPU(gpu_ids=[0])
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

In [8]:
oof_pred_gpu = automl.fit_predict(X_train, roles=roles, verbose=3)

Feats was rejected during automatic roles guess: []
[19:10:50] Layer 1 train process start. Time left 9999999999.45 secs
[19:10:51] Start fitting Lvl_0_Pipe_0_Mod_0_CatBoostGPU ...
[19:10:51] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[19:10:55] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[19:10:59] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_CatBoostGPU (orig) =====


...

[19:11:03] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[19:11:07] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[19:11:11] Lvl_0_Pipe_0_Mod_0_CatBoostGPU fitting and predicting completed
[19:11:11] Time left 9999999978.59 secs

[19:11:11] Layer 1 training completed.



In [9]:
task = Task('reg', metric='mae', device='mgpu')
reader = DictToDaskCudfSeqReader(task=task, seq_params=seq_params, n_jobs=1)

feats = LGBMultiSeqSimpleFeaturesGPU()
model = BoostCBGPU(gpu_ids=[0])
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

In [10]:
oof_pred_mgpu = automl.fit_predict(X_train, roles=roles, verbose=3)

Feats was rejected during automatic roles guess: []
[19:11:12] Layer 1 train process start. Time left 9999999998.92 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/dask/dataframe/core.py:4948: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/dask/dataframe/core.py:4948: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


[19:11:14] Start fitting Lvl_0_Pipe_0_Mod_0_CatBoostGPU ...
[19:11:14] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[19:11:18] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_CatBoostGPU (orig) =====


...

[19:11:26] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[19:11:30] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_CatBoostGPU (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
Default metric period is 5 because MAE is/are not implemented for GPU


[19:11:34] Lvl_0_Pipe_0_Mod_0_CatBoostGPU fitting and predicting completed
[19:11:34] Time left 9999999976.91 secs

[19:11:34] Layer 1 training completed.

